In [29]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# sciKit libraries
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import KFold, cross_val_score
# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line
import folium # map rendering library


In [22]:
cw_data=pd.read_pickle('cw_data.pkl')
cw_data.head()

,Latitude,Longitude,Borough,Neighborhood
0,52.53045,13.29610,Charlottenburg-Wilmersdorf,Charlottenburg-Nord
1,52.50506,13.26206,Charlottenburg-Wilmersdorf,Heerstrasse
2,52.51620,13.25981,Charlottenburg-Wilmersdorf,Westend
3,52.51516,13.29639,Charlottenburg-Wilmersdorf,Schloß Charlottenburg
4,52.52569,13.30473,Charlottenburg-Wilmersdorf,Mierendorffplatz


#### Explore Neighborhoods in CW: Segmenting

In [5]:
cw_venues=pd.read_pickle('cw_venues.pkl')
cw_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Charlottenburg-Nord,52.53045,13.2961,Ristorante Dante,52.526276,13.297480,Italian Restaurant
1,Charlottenburg-Nord,52.53045,13.2961,Belvedere,52.527567,13.295475,Museum
2,Charlottenburg-Nord,52.53045,13.2961,H Tegeler Weg / S Jungfernheide,52.530291,13.296355,Bus Stop
3,Charlottenburg-Nord,52.53045,13.2961,Back Lady,52.530452,13.299959,Bakery
4,Charlottenburg-Nord,52.53045,13.2961,Dreieck Charlottenburg (4) (13),52.533506,13.291440,Intersection


In [7]:
# one hot encoding
cw_onehot = pd.get_dummies(cw_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
cw_onehot['Neighborhood'] = cw_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [cw_onehot.columns[-1]] + list(cw_onehot.columns[:-1])
cw_onehot = cw_onehot[fixed_columns]

cw_onehot.head()

,Neighborhood,ATM,Art Museum,Asian Restaurant,Bakery,Bank,Bar,Bavarian Restaurant,Bistro,Boarding House,Bookstore,Boutique,Brasserie,Breakfast Spot,Buffet,Burger Joint,Bus Stop,Butcher,Café,Chinese Restaurant,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Flower Shop,Food & Drink Shop,Food Truck,French Restaurant,Furniture / Home Store,Gas Station,German Restaurant,Gourmet Shop,Greek Restaurant,Gym,Gym / Fitness Center,Halal Restaurant,Health & Beauty Service,Hookah Bar,Hostel,Hot Spring,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Laser Tag,Lebanese Restaurant,Light Rail Station,Lighting Store,Liquor Store,Mattress Store,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Movie Theater,Moving Target,Museum,Noodle House,Opera House,Organic Grocery,Park,Pastry Shop,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Restaurant,Sandwich Place,Scenic Lookout,Schnitzel Restaurant,Seafood Restaurant,Snack Place,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Sports Club,Stadium,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Swabian Restaurant,Szechuan Restaurant,Tapas Restaurant,Taverna,Tennis Court,Thai Restaurant,Toy / Game Store,Trattoria/Osteria,Turkish Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Shop
0,Charlottenburg-Nord,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Charlottenburg-Nord,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Charlottenburg-Nord,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Charlottenburg-Nord,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Charlottenburg-Nord,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [8]:
cw_onehot.shape

(511, 128)

In [9]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
cw_grouped = cw_onehot.groupby('Neighborhood').mean().reset_index()
cw_grouped.head()

,Neighborhood,ATM,Art Museum,Asian Restaurant,Bakery,Bank,Bar,Bavarian Restaurant,Bistro,Boarding House,Bookstore,Boutique,Brasserie,Breakfast Spot,Buffet,Burger Joint,Bus Stop,Butcher,Café,Chinese Restaurant,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Flower Shop,Food & Drink Shop,Food Truck,French Restaurant,Furniture / Home Store,Gas Station,German Restaurant,Gourmet Shop,Greek Restaurant,Gym,Gym / Fitness Center,Halal Restaurant,Health & Beauty Service,Hookah Bar,Hostel,Hot Spring,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Laser Tag,Lebanese Restaurant,Light Rail Station,Lighting Store,Liquor Store,Mattress Store,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Movie Theater,Moving Target,Museum,Noodle House,Opera House,Organic Grocery,Park,Pastry Shop,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Restaurant,Sandwich Place,Scenic Lookout,Schnitzel Restaurant,Seafood Restaurant,Snack Place,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Sports Club,Stadium,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Swabian Restaurant,Szechuan Restaurant,Tapas Restaurant,Taverna,Tennis Court,Thai Restaurant,Toy / Game Store,Trattoria/Osteria,Turkish Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Shop
0,Barstraße,0.0,0.0,0.0,0.062500,0.0,0.000000,0.0625,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0625,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0625,0.0,0.0625,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0625,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0625,0.125000,0.0625,0.0625,0.0625,0.0,0.000000,0.0,0.062500,0.0625,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0625,0.0,0.0,0.0,0.0625,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000
1,Charlottenburg-Nord,0.0,0.0,0.0,0.200000,0.0,0.000000,0.0000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.200000,0.0,0.000000,0.000000,0.0000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0000,0.0,0.0000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0000,0.000000,0.0000,0.0000,0.0000,0.0,0.000000,0.2,0.200000,0.0000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0000,0.2,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000
2,Düsseldorfer Straße,0.0,0.0,0.0,0.054545,0.0,0.036364,0.0000,0.0,0.018182,0.0,0.036364,0.0,0.0,0.0,0.0,0.000000,0.0,0.036364,0.018182,0.0000,0.018182,0.0,0.0,0.0,0.0,0.036364,0.018182,0.0000,0.0,0.0000,0.0,0.0,0.018182,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.036364,0.018182,0.018182,0.0,0.0,0.0,0.018182,0.0,0.0,0.0000,0.109091,0.0000,0.0000,0.0000,0.0,0.018182,0.0,0.072727,0.0000,0.0,0.000000,0.018182,0.0,0.0,0.0,0.018182,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.018182,0.018182,0.018182,0.0,0.0,0.0,0.0,0.0,0.018182,0.0,0.0,0.0,0.0,0.036364,0.0,0.018182,0.0,0.018182,0.0,0.0,0.0,0.0,0.0,0.0,0.018182,0.018182,0.018182,0.000000,0.0,0.0,0.018182,0.018182,0.036364,0.0,0.0,0.01

In [10]:
cw_grouped.shape

(17, 128)

In [12]:
# print each neighborhood along with the top 2 most common venues
num_top_venues = 2
for hood in cw_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = cw_grouped[cw_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Barstraße----
               venue  freq
0              Hotel  0.12
1  Indian Restaurant  0.06


----Charlottenburg-Nord----
    venue  freq
0  Museum   0.2
1  Bakery   0.2


----Düsseldorfer Straße----
                venue  freq
0               Hotel  0.11
1  Italian Restaurant  0.07


----Forst Grunewald----
                venue  freq
0  Italian Restaurant  0.18
1               Plaza  0.09


----Grunewald----
                venue  freq
0  Italian Restaurant  0.18
1               Plaza  0.09


----Halensee----
         venue  freq
0  Supermarket  0.06
1         Café  0.06


----Heerstrasse----
                venue  freq
0                Café  0.12
1  Italian Restaurant  0.12


----Kantstraße----
                venue  freq
0  Italian Restaurant  0.08
1   German Restaurant  0.06


----Kurfürstendamm----
                venue  freq
0  Italian Restaurant  0.08
1   German Restaurant  0.06


----Mierendorffplatz----
                venue  freq
0         Supermarket  0.14
1  Italian

In [13]:
# put that into a pandas dataframe
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [17]:
# create the new dataframe and display the top 5 venues for each neighborhood
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = cw_grouped['Neighborhood']

for ind in np.arange(cw_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(cw_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(20)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Barstraße,Hotel,Indian Restaurant,Climbing Gym,Gas Station,Moving Target
1,Charlottenburg-Nord,Bus Stop,Bakery,Intersection,Museum,Italian Restaurant
2,Düsseldorfer Straße,Hotel,Italian Restaurant,Vietnamese Restaurant,Bakery,Plaza
3,Forst Grunewald,Italian Restaurant,Jewelry Store,Hotel,Plaza,German Restaurant
4,Grunewald,Italian Restaurant,Jewelry Store,Hotel,Plaza,German Restaurant
5,Halensee,Italian Restaurant,Café,Plaza,Drugstore,Supermarket
6,Heerstrasse,Italian Restaurant,Tennis Court,Stadium,Gas Station,Café
7,Kantstraße,Italian Restaurant,Asian Restaurant,German Restaurant,Vietnamese Restaurant,Korean Restaurant
8,Kurfürstendamm,Italian Restaurant,German Restaurant,Hotel,Café,Boutique
9,Mierendorffplatz,Supermarket,Asian Restaurant,Italian Restaurant,Hotel,Café


#### Explore Neighborhoods in CW: Clustering

In [20]:
# set number of clusters
kclusters = 5

cw_grouped_clustering = cw_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(cw_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 4, 0, 1, 1, 0, 2, 0, 0, 0], dtype=int32)

In [24]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
cw_merged = cw_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
cw_merged = cw_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

cw_merged.head() # check the last columns!

,Latitude,Longitude,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,52.53045,13.29610,Charlottenburg-Wilmersdorf,Charlottenburg-Nord,4,Bus Stop,Bakery,Intersection,Museum,Italian Restaurant
1,52.50506,13.26206,Charlottenburg-Wilmersdorf,Heerstrasse,2,Italian Restaurant,Tennis Court,Stadium,Gas Station,Café
2,52.51620,13.25981,Charlottenburg-Wilmersdorf,Westend,0,Italian Restaurant,Bakery,Park,Café,Bus Stop
3,52.51516,13.29639,Charlottenburg-Wilmersdorf,Schloß Charlottenburg,0,Café,Trattoria/Osteria,German Restaurant,Ice Cream Shop,Art Museum
4,52.52569,13.30473,Charlottenburg-Wilmersdorf,Mierendorffplatz,0,Supermarket,Asian Restaurant,Italian Restaurant,Hotel,Café


In [30]:
# create map
# coordinates of Charlottenburg-Wilmersdorf
latitude  = 52.500000  # neighborhood latitude value
longitude = 13.283333 # neighborhood longitude valu
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(cw_merged['Latitude'], cw_merged['Longitude'], cw_merged['Neighborhood'], cw_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Examine Clusters

In [31]:
# cluster 1
cw_merged.loc[cw_merged['Cluster Labels'] == 0, \
               cw_merged.columns[[1] + list(range(5, cw_merged.shape[1]))]]

,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
2,13.25981,Italian Restaurant,Bakery,Park,Café,Bus Stop
3,13.29639,Café,Trattoria/Osteria,German Restaurant,Ice Cream Shop,Art Museum
4,13.30473,Supermarket,Asian Restaurant,Italian Restaurant,Hotel,Café
5,13.31056,Supermarket,Chinese Restaurant,Italian Restaurant,Bakery,Pet Café
6,13.28370,Hotel,Asian Restaurant,Mexican Restaurant,Boarding House,Doner Restaurant
7,13.31435,Italian Restaurant,Asian Restaurant,German Restaurant,Vietnamese Restaurant,Korean Restaurant
8,13.31891,Italian Restaurant,German Restaurant,Hotel,Café,Boutique
9,13.29569,Italian Restaurant,Café,Plaza,Drugstore,Supermarket
11,13.29005,Café,Italian Restaurant,Bakery,Bank,Ice Cream Shop
13,13.31653,Hotel,Italian Restaurant,Vietnamese Restaurant,Bakery,Plaza


In [34]:
# cluster 2
cw_merged.loc[cw_merged['Cluster Labels'] == 1, \
               cw_merged.columns[[1] + list(range(5, cw_merged.shape[1]))]]

,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
10,13.2658,Italian Restaurant,Jewelry Store,Hotel,Plaza,German Restaurant
16,13.2658,Italian Restaurant,Jewelry Store,Hotel,Plaza,German Restaurant
